# Combined Model incorporating Drafted/Undrafted Model (Step 4) 

This model will have the Box score data and the autostats percentage data as well as incorporating the model created for drafted/undrafted using probabilities

In [ ]:
import pandas as pd 
import numpy as np 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import glm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LassoCV
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# reading in the data for the model
draft_data = pd.read_csv("combined_data.csv").drop('Unnamed: 0', axis=1)

In [ ]:
draft_data = draft_data.fillna(0)

In [ ]:
# splits data into X and Y 
X_full = draft_data.drop(['Drafted', 'PICK_NUMBER', 'AutoStats Games', 'PICK_NUMBER_x', 'PLAYER_GAMES','Offensive Chances', 'Defensive Chances'], axis=1)
y_full = draft_data[['Drafted']]

## Converting Birth Date to Age

In [ ]:
# reading in the data 
roster = pd.read_csv("Roster Decode.csv")
pick = pd.read_csv("NBA Caliber Players.csv")

In [ ]:
seasons = pick[['PLAYER_ID', 'START_SEASON']]

In [ ]:
## grabbing only the columns needed 
df = roster[['PLAYER_ID', 'HEIGHT','WEIGHT', 'BIRTH_DATE', 'POSITION_ABBREV']]

In [ ]:
print(len(df))
print(len(roster))

In [ ]:
df = pd.merge(df, seasons, how='inner', on='PLAYER_ID')

In [ ]:
df['BIRTH_DATE'].isna().sum()

In [ ]:
## splitting the birth_date
df['YEAR'] = df['BIRTH_DATE'].str.split(pat = '-', expand = True)[2]

In [ ]:
df['YEAR'] = df['YEAR'].fillna(0)
df['START_SEASON'] = df['START_SEASON'].fillna(0)

In [ ]:
## converting column to an integer
df['YEAR'] = df['YEAR'].astype('int')

birth_date - july 1st of start year 

In [ ]:
## creating a list of years 
years = []
for i in df['YEAR']: 
    if i == 0: 
        years.append(i)
    elif i > 23: 
        years.append(i+1900) 
    elif i <= 23: 
        years.append(i+2000)

In [ ]:
# adding this list into the dataframe
df['YEAR'] = years

In [ ]:
df['Age'] = df['START_SEASON'] - df['YEAR']

In [ ]:
a = df[(df['Age'] < 57) & (df['Age'] > 0)]
a['Age'].mean()

In [ ]:
## filling in Age NAs
df.loc[df["Age"] > 56, "Age"] = a['Age'].mean()
df.loc[df['Age'] <= 0, "Age"] = a['Age'].mean()

In [ ]:
## getting the age of each player based on year 

# import pandas as pd
# from datetime import datetime, date
  
# # This function converts given date to age
# def age(born):
#     born = datetime.strptime(born, "%Y").date()
#     today = date.today()
#     return today.year - born.year 

In [ ]:
# ## appling the function
# df['Age'] = df['YEAR'].apply(age)

In [ ]:
## dropping unneeded columns
df.drop(['BIRTH_DATE', 'YEAR'], axis=1, inplace=True)

In [ ]:
## dropping duplicate players
df = df.drop_duplicates(subset='PLAYER_ID', keep="first")

In [ ]:
## drop people who started before 2012
df = df[df['START_SEASON'] >= 2012]

In [ ]:
df.rename({'POSITION_ABBREV': 'Pos'}, axis=1, inplace=True)

In [ ]:
df['Pos']=np.where(df['Pos']=='G','Guard',
                                    np.where(df['Pos']=='PG','Guard',
                                    np.where(df['Pos']=='SF','Wing',   
                                    np.where(df['Pos']=='PF','Big',         
                                    np.where(df['Pos']=='G/F','Wing',
                                    np.where(df['Pos']=='F',
                                    np.where(df['HEIGHT']<=79,'Wing','Big'),
                                    np.where(df['Pos']=='SG',
                                    np.where(df['HEIGHT']<=75,'Guard','Wing'),          
                                    np.where(df['Pos']=='F/C','Big',
                                    np.where(df['Pos']=='C','Big',
                                    np.where(df['HEIGHT']<=75,'Guard',
                                    np.where(df['HEIGHT']>79,'Big','Wing')))))))))))

In [ ]:
## Finding averages to replace NAs
df.groupby('Pos').agg({'HEIGHT': 'mean', 'WEIGHT': 'mean'})

In [ ]:
## replacing Height and Weight NAs
df['HEIGHT'] = df.groupby('Pos')['HEIGHT'].apply(lambda x: x.fillna(x.mean()))
df['WEIGHT'] = df.groupby('Pos')['WEIGHT'].apply(lambda x: x.fillna(x.mean()))


In [ ]:
X_full = pd.merge(X_full, df, on = ['PLAYER_ID'], how='inner')

In [ ]:
X_full.drop(['Pos','START_SEASON', 'Age'], axis=1, inplace=True)

## Drafted/Undrafted Model - Training
1. Normalize Data 
2. Drop features from RFE 
3. Oversample 
4. Hypertune  
5. Random forest 

### Normalize and Split

In [ ]:
# X_full = X_full.drop(['League'], axis=1)

In [ ]:
#X_full = (X_full-X_full.mean())/(X_full.max() - X_full.min())

In [ ]:
# splitting with new data
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, random_state=0)

In [ ]:
X_train.isna().sum().sum()

### PCA

In [ ]:
## Normalize the data 
X_full=(X_full-X_full.mean())/(X_full.max()-X_full.min())

In [ ]:
# splitting with new data
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, random_state=0)

y_train.reset_index(drop=True, inplace=True) 
y_test.reset_index(drop=True, inplace=True)
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

In [ ]:
pca_all = PCA(n_components=153)
pca_all.fit(X_full)

In [ ]:
plt.grid()
plt.plot(np.cumsum(pca_all.explained_variance_ratio_ * 100))
plt.xlabel('Number of components')
plt.ylabel('Explained variance')

In [ ]:
# train and test sets
pca_10 = PCA(n_components=60)
pca_10.fit(X_train)
X_train_reduced = pca_10.transform(X_train)
X_test_reduced = pca_10.transform(X_test)

# verify shape after PCA
print("Shape:", X_train_reduced.shape)
print("Test images shape: ", X_test_reduced.shape)

# get exact variability retained
print("\nVar retained (%):", 
      np.sum(pca_10.explained_variance_ratio_ * 100))

### Recursive Feature Embedding

In [ ]:
from sklearn.feature_selection import RFECV

rfc = RandomForestClassifier()


RFE_model = RFECV(
    estimator = rfc, # the ML model
    scoring = 'accuracy',  
    cv=2)

In [ ]:
RFE_model.fit(X_train_reduced, y_train)


In [ ]:
mask = RFE_model.get_support()
features = np.array(X_train.columns) 
best_features = features[mask]
 
print("All features: ", X_train.shape[1])
#print(features)

print("Selected best: ", best_features.shape[0])
#print(features[mask]) 

In [ ]:
for i in features[mask]:
    print("'" + i + "',")

In [ ]:
cols = ['PLAYER_ID','Offensive Points/Chance',
'Defensive Points/Chance',
'Defensive Transition%',
'Transition Touch%',
'Transition Points/Touch',
'AutoStats FG%',
'Contested Jumper FG%',
'% of Jumpers Contested',
'Catch & Shoot Jumper FG%',
'% of Shots at Rim',
'Average Influence Score',
'AutoStats FG% Defended',
'Uncontested Jumper FG% Defended',
'% of Defended 3s Contested',
'Layup FG% Defended',
'Led to Shot %',
'Assist %',
'FT Assist %',
'% of Offensive Rebounds Contested',
'Defensive Adjusted Rebound %',
'Defensive Contested Rebound %',
'Crash & Boxout %',
'Drives/Touch',
'Player Points/Drive',
'Drives Defended/Defensive Chances',
'Team Points/Drive Chance Allowed',
'Isos/Touch',
'Isos Defended/Defensive Chances',
'Team Points/Iso Chance Allowed',
'SC- Screens/Chance',
'SC Def- Screens/Chance',
'SC Def- Team Points/Chance',
'Closeout Touch %',
'Closeouts/Chance',
'Points/Closeout Allowed',
'Full Closeout %',
'Cutter- Team Points/Chance',
'Cutter- Touches/Screen',
'Cutter- Points/Touch',
'Screener- Cutter Touches/Screen',
'Screener- Cutter Points/Touch',
'Screener- Screener Points/Touch',
'Cutter Def- Screens/Chance',
'Cutter Def- Team Points/Chance',
'Cutter Def- Cutter Touches/Screen',
'Cutter Def- Screener Points/Touch',
'Screener Def- Screens/Chance',
'Screener Def- Cutter Touch Points',
'Screener Def- Screener Points/Touch',
'Setter- Handoffs/Touch',
'Receiver Defender- Team Points/Chance',
'Setter Defender- Handoffs/Chance',
'Setter Defender- Team Points/Chance',
'FGM_30',
'FGA_30',
'TPM_30',
'TPA_30',
'FTM_30',
'FTA_30',
'FOULS_30',
'STEALS_30',
'ASSISTS_30',
'OREB_30',
'DREB_30',
'TOV_30',
'FG_PCT',
'TP_PCT',
'FT_PCT',
'USG',
'BLOCK_PCT',
'OREB_PCT',
'DREB_PCT',
'FTM_Rate',
'STL_PCT',
'TOV_PCT',
'HEIGHT']

In [ ]:
X_train = X_train[cols]
X_test = X_test[cols]

In [ ]:
# encode labels as integers for y_train 
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y_train = encoder.transform(y_train)

In [ ]:
# encode labels as integers for y_train 
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y_test = encoder.transform(y_test)

In [ ]:
encoder.classes_

In [ ]:
from imblearn.combine import SMOTETomek

counter = Counter(encoded_Y_train)
for k,v in counter.items():
    per = v / len(encoded_Y_train) * 100
    print('Class=%s, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.title('Data Counts before Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

smote_tomek = SMOTETomek(random_state=888)
X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train_reduced.drop(['PLAYER_ID'], axis=1), encoded_Y_train)

counter = Counter(y_train_resampled)
for k,v in counter.items():
    per = v / len(y_train_resampled) * 100
    print('Class=%s, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.title('Data Counts before Resampling')
pyplot.bar(counter.keys(), counter.values(), color = 'grey')
pyplot.show()

In [ ]:
from imblearn.combine import SMOTETomek

fig, ax = plt.subplots(figsize = (5,4))

counter = Counter(encoded_Y_test)
for k,v in counter.items():
    per = v / len(encoded_Y_test) * 100
    print('Class=%s, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
ax.set_xticks([0,1])
ax.set_xticklabels(('Undrafted', 'Drafted'))
pyplot.title('Data Counts before Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

smote_tomek = SMOTETomek(random_state=888)
X_test_resampled, y_test_resampled = smote_tomek.fit_resample(X_test.drop(['PLAYER_ID'], axis=1), encoded_Y_test)


fig, ax = plt.subplots(figsize = (5,4))

counter = Counter(y_test_resampled)
for k,v in counter.items():
    per = v / len(y_test_resampled) * 100
    print('Class=%s, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
ax.set_xticks([0,1])
ax.set_xticklabels(('Undrafted', 'Drafted'))
pyplot.title('Data Counts after Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

### Random Forest 

In [ ]:
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

In [ ]:
grid_search = GridSearchCV(RandomForestClassifier(),
                           param_grid=param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_estimator_)

max_depth=9, max_features=None, max_leaf_nodes=9,
                       n_estimators=50

In [ ]:
# Instantiate and fit the RandomForestClassifier
forest = RandomForestClassifier(max_depth=3, max_features=None, 
                                max_leaf_nodes=6, n_estimators=25)
forest.fit(X_train, y_train)


In [ ]:
# Use the forest's predict method on the test data
predictions = forest.predict(X_train)

# View accuracy score
accuracy_score(y_train, predictions)

# Evaluation metrics
print("**Train Classification Report**")
print(classification_report(y_train, predictions))

# Use the forest's predict method on the test data
predictions = forest.predict(X_test)

# View accuracy score
accuracy_score(y_test, predictions)

# Evaluation metrics
print("**Test Classification Report**")
print(classification_report(y_test, predictions))

In [ ]:
predictions[1]

In [ ]:
## The first column is 0 and second column is 1 
## 0 = 'No'
## 1 = 'Yes'
probs = forest.predict_proba(X_test)
len(probs[:,0])


In [ ]:
probs.shape

## Drafted/Undrafted Model - Implementation of Data

Will input the entire dataset to then get probabilities and implement those probabilities into the overall rank model

In [ ]:
## still need to drop the features not involved in the data above 
X_full = X_full[cols]

In [ ]:
player_id = X_full['PLAYER_ID']

In [ ]:
X_full.drop(['PLAYER_ID'], axis=1, inplace=True)
probs = forest.predict_proba(X_full)
predictions = forest.predict(X_full)

In [ ]:
predictions[10]

In [ ]:
probs[10]

In [ ]:
# Visualization for presentation 
prob_df = pd.DataFrame(probs, columns = ['Undraft_Prob', 'Draft_Prob'])
prob_df['PLAYER_ID'] = player_id

names = pd.read_csv('NBA Caliber Players.csv')
names = names[['PLAYER_ID', 'PLAYER']]

probabilities = pd.merge(names, prob_df, on='PLAYER_ID', how='inner')

# probabilities.set_index('PLAYER').drop(['PLAYER_ID'], axis=1).iloc[350:400]

In [ ]:
probabilities[probabilities['PLAYER_ID'].isin([1349630, 1373356])].set_index('PLAYER').drop(['PLAYER_ID'], axis=1)

In [ ]:
undrafted_prob = list(probs[:,0])

In [ ]:
len(undrafted_prob)

**Now, we can implement this array into the data as a new feature for implementation in the next model predicting draft rank**

In [ ]:
draft_data['undraft_prob'] = undrafted_prob

In [ ]:
#draft_data = draft_data.rename({'PICK_NUMBER_x' : 'PICK_NUMBER'}, axis=1)

In [ ]:
draft_data.to_csv("incorporated_data.csv")

In [ ]:
# splits data into X and Y 
X = draft_data.drop(['Drafted', 'PLAYER_ID', 'PICK_NUMBER', 'GAMES_PLAYED', 'PICK_NUMBER_x', 'AutoStats Games', 'League ID_y'], axis=1)
y = draft_data[['PICK_NUMBER']]

In [ ]:
len(X.columns)

In [ ]:
# Normalize the data to be between -1 and 1 (mean centered)
X = (X-X.mean())/(X.max() - X.min())

In [ ]:
# splitting with new data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify = y)

## Recursive Feature Embedding 

In [ ]:
rfc = RandomForestClassifier()


RFE_model = RFECV(
    estimator = rfc, # the ML model
    scoring = 'accuracy',  
    cv=2)

In [ ]:
RFE_model.fit(X_train, y_train)

In [ ]:
mask = RFE_model.get_support()
features = np.array(X_train.columns) 
best_features = features[mask]
 
print("All features: ", X_train.shape[1])

print("Selected best: ", best_features.shape[0])

In [ ]:
for i in features[mask]:
    print(i)

In [ ]:
# cols = ['Offensive Chances',
# 'Defensive Chances',
# 'Touches',
# 'Defensive Points/Chance',
# 'Defensive Transition%',
# 'Transition Touch%',
# 'AutoStats FGM',
# 'AutoStats FGA',
# 'Contested Jumper FGM',
# 'Contested Jumper FGA',
# '% of Jumpers Contested',
# 'Catch & Shoot Jumper FG%',
# 'Average Influence Score',
# 'Expected eFG%',
# 'AutoStats FGA Defended',
# 'AutoStats FG% Defended',
# 'AutoStats 3P% Defended',
# 'Contested Jumper FGA Defended',
# 'Contested Jumper FG% Defended',
# 'Uncontested Jumper FG% Defended',
# 'Contested 3P% Defended',
# 'Uncontested 3P% Defended',
# 'Average Influence Score Defended',
# 'Expected eFG% Defended',
# 'Expected eFG% Defended- 3s',
# 'Layup FG% Defended',
# 'Passes/Touch',
# 'Assist %',
# 'Secondary Assist %',
# 'AutoStats Rebounds',
# 'Rebound Chances',
# 'Rebound %',
# 'Adjusted Rebound %',
# 'Contested Rebound %',
# 'Offensive Rebound %',
# 'Offensive Adjusted Rebound %',
# 'Offensive Contested Rebound %',
# 'AutoStats Defensive Rebounds',
# 'Defensive Rebound Chances',
# 'Crash %',
# 'Rebound at Rim %',
# 'Drives Defended',
# 'Player Points/Drive Allowed',
# 'Team Points/Drive Chance Allowed',
# 'Direct Drives Defended %',
# 'Player Points/Direct Drive Allowed',
# 'Team Points/Direct Drive Chance Allowed',
# 'Drive Defended Start-Iso %',
# 'Isos Defended',
# 'Isos Defended/Defensive Chances',
# 'Direct Isos Defended %',
# 'Player Points/Direct Iso Allowed',
# 'Team Points/Direct Iso Chance Allowed',
# 'BH Def- Team Points/Chance',
# 'BH Def- Direct Screen %',
# 'SC Def- Screens',
# 'SC Def- Ball Handler Points',
# 'SC Def- Team Points',
# 'SC Def- Team Points/Chance',
# 'SC Def- Team Points/Direct Chance',
# 'SC Def- Ball Handler Pass to Screener %',
# 'SC Def- Up to Touch %',
# 'SC Def- Switch %',
# 'Points/Closeout Touch- Offense',
# 'Direct Closeout Touch %',
# 'Points/Full Closeout Touch- Offense',
# 'Team Points/Full Closeout Chance- Offense',
# 'Closeout Points Allowed',
# 'Points/Closeout Allowed',
# 'Team Points/Closeout Chance Allowed',
# 'Points/Direct Closeout Touch Allowed',
# 'Team Points/Direct Closeout Chance Allowed',
# 'Short Closeout %',
# 'Cutter- Screens/Chance',
# 'Cutter- Direct Cutter Touch %',
# 'Screener- Screens/Chance',
# 'Screener- Team Points/Chance',
# 'Screener- Cutter Touches/Screen',
# 'Screener- Team Points/Screener Touch Chance',
# 'Cutter Def- Cutter Touches/Screen',
# 'Cutter Def- Team Points/Cutter Touch Chance',
# 'Cutter Def- Screener Points/Touch',
# 'Cutter Def- Team Points/Screener Touch Chance',
# 'Cutter Def- Gap %',
# 'Cutter Def- Switch %',
# 'Screener Def- Screens',
# 'Screener Def- Team Points/Chance',
# 'Screener Def- Cutter Touches/Screen',
# 'Screener Def- Cutter Touch Points',
# 'Screener Def- Team Points/Cutter Touch Chance',
# 'Screener Def- Screener Touches/Screen',
# 'Screener Def- Screener Points/Touch',
# 'Screener Def- None %',
# 'Screener Def- Up to Touch %',
# 'Screener Def- Switch %',
# 'Setter- Handoffs/Touch',
# 'Receiver Defender- Handoff Pick %',
# 'Setter Defender- Handoffs',
# 'Setter Defender- Handoffs/Chance',
# 'Setter Defender- Receiver Points/Handoff',
# 'Setter Defender- Team Points',
# 'Setter Defender- Team Points/Chance',
# 'Setter Defender- Team Points/Pick Chance',
# 'Setter Defender- Receiver Points/Direct Handoff',
# 'AutoStats Games Rank',
# 'Offensive Chances Rank',
# 'Defensive Points/Chance Rank',
# 'Defensive Transition% Rank',
# 'Transition Touch% Rank',
# 'AutoStats FGM Rank',
# 'Contested Jumper FGM Rank',
# 'Contested Jumper FGA Rank',
# 'Catch & Shoot Jumper FGM Rank',
# 'FT Assists Rank',
# 'AutoStats Rebounds Rank',
# 'Rebound Chances Rank',
# 'AutoStats Defensive Rebounds Rank',
# 'Defensive Rebound Chances Rank',
# 'Iso Player Points Allowed Rank',
# 'Iso Team Points Allowed Rank',
# 'Cutter- Screens/Chance Rank',
# 'Screener- Screens Rank',
# 'Screener- Screens/Chance Rank',
# 'Screener Def- Screens/Chance Rank',
# 'Screener Def- Cutter Touch Points Rank',
# 'Setter- Handoffs Rank',
# 'Setter- Handoffs/Touch Rank',
# 'Offensive Chances Role Rank',
# 'Defensive Chances Role Rank',
# 'Offensive Transition% Role Rank',
# 'Transition Touch% Role Rank',
# 'AutoStats FGM Role Rank',
# 'AutoStats FGA Role Rank',
# 'Contested Jumper FGM Role Rank',
# 'Contested Jumper FGA Role Rank',
# 'Catch & Shoot Jumper FGM Role Rank',
# 'FT Assists Role Rank',
# 'AutoStats Rebounds Role Rank',
# 'Rebound Chances Role Rank',
# 'AutoStats Defensive Rebounds Role Rank',
# 'Defensive Rebound Chances Role Rank',
# 'Drive Player Points Role Rank',
# 'Drives Defended Role Rank',
# 'Drive Player Points Allowed Role Rank',
# 'Iso Team Points Role Rank',
# 'Isos Defended Role Rank',
# 'Iso Player Points Allowed Role Rank',
# 'BH Def- Team Points Role Rank',
# 'SC- Screens/Chance Role Rank',
# 'Cutter- Screens/Chance Role Rank',
# 'Cutter- Touch Points Role Rank',
# 'Screener- Screens/Chance Role Rank',
# 'Screener Def- Screens/Chance Role Rank',
# 'Screener Def- Cutter Touches Role Rank',
# 'Screener Def- Cutter Touch Points Role Rank',
# 'Receiver- Handoffs/Touch Role Rank',
# 'Setter- Handoffs/Touch Role Rank',
# 'Setter- Team Points Role Rank',
# 'MINUTES_PLAYED',
# 'FIELD_GOALS_MADE',
# 'FIELD_GOALS_ATT',
# 'FREE_THROWS_MADE',
# 'FREE_THROWS_ATT',
# 'THREE_POINT_ATT',
# 'TOTAL_POINTS',
# 'BLOCKS',
# 'PERSONAL_FOULS',
# 'HIGH_POINTS_GAME',
# 'DOUBLE_DOUBLES',
# 'WINS',
# 'LOSSES'] 

In [ ]:
X_train = X_train[cols]
X_test = X_test[cols]

## Artifical Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import keras

In [ ]:
# Convert X values to numpy array 
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)

In [ ]:
# encode labels as integers for y_train 
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y_train = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_train = np_utils.to_categorical(encoded_Y_train)

In [ ]:
list(encoder.classes_)

In [ ]:
# encode labels as integers for y_test 
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y_test = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y_test = np_utils.to_categorical(encoded_Y_test)

In [ ]:
list(encoder.classes_)

### Weighting the Model

In [ ]:
## Invert the value_counts to get the weights 
1/y_train.value_counts()*100

In [ ]:
calc_weights = {0:4.166667, 1:2.857143, 2:1.265823, 3:0.617284, 4:4.347826, 5:4.166667, 6:0.164745}

In [ ]:
from sklearn.utils import class_weight
class_w = class_weight.compute_class_weight('balanced',
                                                 classes = ['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other- NBA'],
                                                 y = y_train.values.reshape(-1))

In [ ]:
class_w

In [ ]:
class_weights = {0: 5.67857143, 1: 3.89387755, 2:1.72513562, 3:0.84126984, 4:5.92546584, 5:5.67857143, 6:0.22452342}

### Or Oversampling the Model

In [ ]:
encoded_Y_train.shape

In [ ]:
dummy_y_train.shape

In [ ]:
X_train_array.shape

In [ ]:
from imblearn.combine import SMOTETomek

counter = Counter(encoded_Y_train)
for k,v in counter.items():
    per = v / len(dummy_y_train) * 100
    print('Class=%s, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.title('Data Counts before Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

smote_tomek = SMOTETomek(random_state=888)
X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train_array, encoded_Y_train)

counter = Counter(y_train_resampled)
for k,v in counter.items():
    per = v / len(y_train_resampled) * 100
    print('Class=%s, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.title('Data Counts before Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

In [ ]:
from imblearn.combine import SMOTETomek

counter = Counter(encoded_Y_test)
for k,v in counter.items():
    per = v / len(encoded_Y_test) * 100
    print('Class=%s, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.title('Data Counts before Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

smote_tomek = SMOTETomek(random_state=888)
X_test_resampled, y_test_resampled = smote_tomek.fit_resample(X_test_array, encoded_Y_test)

counter = Counter(y_test_resampled)
for k,v in counter.items():
    per = v / len(y_test_resampled) * 100
    print('Class=%s, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.title('Data Counts before Resampling')
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

In [ ]:
## change the resampled data to dummy data 
y_train_resampled = np_utils.to_categorical(y_train_resampled)

y_test_resampled = np_utils.to_categorical(y_test_resampled)

### ANN #1

In [ ]:
# build a model
model = Sequential()
model.add(Dense(50, input_shape=(X_train_array.shape[1],), activation='relu')) # input shape is (features,)
model.add(Dense(25, activation='softmax'))
model.add(Dense(16, activation='softmax'))
model.add(Dense(7, activation='softmax'))
model.summary()

# compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', # this is different instead of binary_crossentropy (for regular classification)
              metrics=['accuracy'])

In [ ]:
# early stopping callback
# This callback will stop the training when there is no improvement in  
# the validation loss for 10 consecutive epochs.  
es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                   mode='min',
                                   patience=10, 
                                   restore_best_weights=True) 

# now we just update our model fit call
history =  model.fit(X_train_array,
                    dummy_y_train,
                    callbacks=[es],
                    epochs=1000, 
                    class_weight = calc_weights,
                    batch_size=5,
                    shuffle=True,
                    validation_split=0.2,
                    verbose=1)

In [ ]:
history_dict = history.history

# learning curve
# accuracy
acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']

# loss
loss = history_dict['loss']
val_loss = history_dict['val_loss']

# range of X (no. of epochs)
epochs = range(1, len(acc) + 1)

# plot
# "r" is for "solid red line"
plt.plot(epochs, acc, 'r', label='Training accuracy')
# b is for "solid blue line"
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

preds = model.predict(X_train_array) # see how the model did!
print(preds[4]) # i'm spreading that prediction across three nodes and they sum to 1
print(np.sum(preds[0])) # sum it up! Should be 1

In [ ]:
predictions = preds.argmax(axis=1)

In [ ]:
cm = confusion_matrix(encoded_Y_train, predictions)

In [ ]:
len(y_train)

In [ ]:
len(encoded_Y_train)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other']); ax.yaxis.set_ticklabels(['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other']);

In [ ]:
y_train.value_counts()

### ANN #2

In [ ]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense
# Neural network
model2 = Sequential()
model2.add(Dense(16, input_dim=23, activation='relu'))
model2.add(Dense(12, activation='relu'))
model2.add(Dense(7, activation='softmax'))

In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history2 = model2.fit(X_train_array, dummy_y_train, 
                      epochs=100, 
                      class_weight = class_weights, 
                      batch_size=32)

In [ ]:
y_pred = model2.predict(X_test_array)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(dummy_y_test)):
    test.append(np.argmax(dummy_y_test[i]))

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

In [ ]:
history2 = model2.fit(X_train_array, dummy_y_train, 
                      validation_data = (X_test_array, dummy_y_test), 
                      epochs=100, 
                      #class_weight = class_weights, 
                      batch_size=32)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
X_test_array.shape

In [ ]:
len(encoded_Y_test)

In [ ]:
pred = model2.predict(X_test_array)

predictions = pred.argmax(axis=1)

cm = confusion_matrix(encoded_Y_test, predictions)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt     

ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other']); ax.yaxis.set_ticklabels(['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other']);

Not really predicting 9-12 but does predict the next class!?

In [ ]:
predictions[21]

In [ ]:
pred[21]

In [ ]:
encoded_Y_test[21]

In [ ]:
print(classification_report(encoded_Y_test, predictions))

### Tunning ANN

In [ ]:
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.constraints import MaxNorm


# Function to create model, required for KerasClassifier
def create_model(neurons):
    # create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=171, kernel_initializer='glorot_normal',  activation='softsign', kernel_constraint=MaxNorm(4)))
    # model.add(Dense(12, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))
    model.add(Dense(7, activation='softmax', kernel_initializer='glorot_normal'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
model = KerasClassifier(model=create_model, epochs = 50, batch_size = 100, verbose=0)
# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# epochs = [10, 50, 100]
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

# learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
#momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

# init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
# weight_constraint = [1.0, 2.0, 3.0, 4.0, 5.0]
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(model__neurons=neurons)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_array, dummy_y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.655136 using {'batch_size': 100, 'epochs': 50}

Best: 0.643606 using {'optimizer__learning_rate': 0.001}


Best: 0.656184 using {'model__optimizer': 'Nadam'}

Best: 0.651992 using {'model__init_mode': 'glorot_normal'}

Best: 0.649895 using {'model__activation': 'softsign'}

Best: 0.656184 using {'model__dropout_rate': 0.2, 'model__weight_constraint': 4.0}

Best: 0.642558 using {'model__neurons': 30 or 15 }




### Trying with new parameters

In [ ]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Neural network
model3 = Sequential()
model3.add(Dense(15, input_dim=171, activation='relu'))
model3.add(Dense(12, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(7, activation='softmax'))

In [ ]:
model3.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
history3 = model3.fit(X_train_array, dummy_y_train, 
                      epochs=50, 
                      class_weight = class_weights, 
                      batch_size=100)

In [ ]:
y_pred = model3.predict(X_test_array)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(dummy_y_test)):
    test.append(np.argmax(dummy_y_test[i]))

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

In [ ]:
history3 = model3.fit(X_train_array, dummy_y_train, 
                      validation_data = (X_test_array, dummy_y_test), 
                      epochs=100, 
                      class_weight = class_weights, 
                      batch_size=32)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history3.history['accuracy'])
plt.plot(history3.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
pred = model3.predict(X_test_array)

predictions = pred.argmax(axis=1)

cm = confusion_matrix(encoded_Y_test, predictions)

In [ ]:
print(classification_report(encoded_Y_test, predictions))

In [ ]:
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other']); ax.yaxis.set_ticklabels(['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other']);

## Problems 
ANNs using weights or oversampling does not guess one of the classes 

Second ANN does better than first with weights 

## Penalized-SVM

In [ ]:
# load library
from sklearn.svm import SVC
import sklearn.metrics

# we can add class_weight='balanced' to add panalize mistake
svc_model = SVC(class_weight='balanced', probability=True)

svc_model.fit(X_train, y_train)

svc_predict = svc_model.predict(X_test)# check performance
#print('ROCAUC score:',roc_auc_score(y_test, svc_predict))
print('Accuracy score:',accuracy_score(y_test, svc_predict))
#print('F1 score:',f1_score(y_test, svc_predict))

In [ ]:
print(classification_report(y_test, svc_predict))

In [ ]:
cm_svm = confusion_matrix(y_test, svc_predict)

In [ ]:
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other']); ax.yaxis.set_ticklabels(['1-4', '13-18', '19-30', '31-60', '5-8', '9-12', 'Other']);